# Tratamento da tabela resultados

## Instalando e importando bibliotecas

In [ ]:
pip install ydata-profiling matplotlib ipywidgets pandas pyarrow

In [2]:
import pandas as pd
import pyarrow
import matplotlib.pyplot as plt

## Importando dataset

In [3]:
df_resultados = pd.read_parquet("RESULTADOS_04_06_2024_full.parquet", engine='pyarrow')

## Tratando dados

#### Tirando coluna vazia

In [4]:
df_resultados = df_resultados.drop(columns=['Unnamed: 0'])

In [5]:
df_resultados = df_resultados.drop_duplicates()

#### Removendo nulos

In [6]:
df_resultados = df_resultados.dropna()

#### Convertendo data de object pra datetime

In [7]:
df_resultados['DATA'] = pd.to_datetime(df_resultados['DATA'], errors='coerce')

#### Criando as colunas por status e por id

In [8]:
pivot_df = df_resultados.pivot_table(index='KNR', columns=["ID", "STATUS"], aggfunc='size', fill_value=0)

pivot_df.columns = [f'QTD_STATUS_{col[0]}_OK' if col[1]== 10 else f'QTD_STATUS_{col[0]}_NOK' for col in pivot_df.columns]

pivot_df.reset_index(inplace=True)

#### Criando coluna de quantidade de operações por KNR, que, no caso, também é a soma dos status de cada resultado

In [9]:
pivot_df["TEMPO_MEDIO"] = df_resultados.groupby('KNR').DATA.transform('max') - df_resultados.groupby('KNR').DATA.transform('min')

### Transformando tempo em minutos

In [10]:
pivot_df["TEMPO_MEDIO"] = pivot_df["TEMPO_MEDIO"].dt.total_seconds()/60

In [13]:
pivot_df.head(20)

,KNR,QTD_STATUS_1_OK,QTD_STATUS_1_NOK,QTD_STATUS_2_OK,QTD_STATUS_2_NOK,QTD_STATUS_718_OK,QTD_STATUS_718_NOK,TEMPO_MEDIO
0,2023-2056234,5,0,766,4,71,0,NaN
1,2024-0576004,0,0,0,0,4,0,NaN
2,2024-0576005,0,0,0,0,5,0,NaN
3,2024-0576006,0,0,0,0,4,0,NaN
4,2024-0576008,0,0,6,0,32,0,NaN
5,2024-0576015,0,0,0,0,5,1,NaN
6,2024-0576016,0,0,0,0,1,4,6553.933333
7,2024-0576017,0,0,0,0,5,2,6553.933333
8,2024-0576018,0,0,4,0,32,0,6553.933333
9,2024-1176003,0,0,0,0,4,0,6553.933333


## Exportando em parquet para pré processamento e modelagem

In [12]:
pivot_df.to_parquet("RESULTADOS_04_06_2024_TRATADO.parquet")